In [1]:
from improved_find_data_start_row import *
from financials_clean_year_assign import * 
import expertForecastFunctionsCopy1 as ff




In [2]:
financial_data=improved_load_excel_with_autodetect("/Users/myself/Desktop/Walmart USA Serching for Growth/walmartHistoricalFinancials.xlsx")

financial_data=clean_cashflow_data(financial_data, 3)
financial_data=update_column_names(financial_data, base_year=2021,index_name = 'Financial Metric')



In [4]:
categories, base_mapping = ff.base_mapping(financial_data)
# assumptions = {
#     'Sales': {'type': 'YOY', 'rates': [0.0814, 0.0392, 0.0261, 0.0244, 0.0237, 0.0238]},
#     'Costs of goods sold': {'type': '% of revenue', 'rates': [0.7541, 0.7513, 0.7513, 0.7513, 0.7513, 0.7513]},
#     'Gross profit': {'type': 'direct', 'rates': None},  # Calculated directly as difference
#     'Operating and SG&A costs': {'type': '% of sales', 'rates': [0.1863, 0.1834, 0.1834, 0.1834, 0.1834, 0.1834]}}


assumptions = {
    'Operating revenue (Turnover)': {'type': 'YOY', 'rates': [0.015]},
    'Gross Profit': {'type': 'YOY', 'rates': [0.014]},
    'EBITDA': {'type': 'YOY', 'rates': [-0.022]},
    'Cash flow [Net Income before D&A]]': {'type': 'YOY', 'rates': [-0.046]},
    'Operating profit (loss) [EBIT]': {'type': 'YOY', 'rates': [-0.02]},
    'Profit (loss) for the period [Net income]': {'type': 'YOY', 'rates': [-0.019]}
}


# Base year for forecast
base_year = '2021'
forecast_years = ['2022F']


yoy_forecast_df = ff.forecast_financials(financial_data, assumptions, base_year, forecast_years)

#financial_data = ff.handle_missing_values(financial_data)
yoy_forecast_df, financial_data

,2022F,CAGR
Financial Metric,,
Non-current assets,NaN,0.0
Intangible assets,NaN,0.0
Tangible fixed assets,NaN,0.0
Other non-current assets,NaN,0.0
Current assets,NaN,0.0
Stock,NaN,0.0
Debtors,NaN,0.0
Other current assets,NaN,0.0
Of which cash and cash equivalent,NaN,0.0


In [4]:
historical_regression_forecast = ff.linear_regression_forecast(financial_data, forecast_years)
result_forecast, result_percentages = ff.ratio_forecast_regression(financial_data, historical_regression_forecast, base_mapping, forecast_years)

result_forecast

,Non-current assets,Intangible assets,Tangible fixed assets,Other non-current assets,Current assets,Other current assets,Total assets,Net current assets,Shareholders funds,Other shareholders funds,...,Profit (loss) after tax [PAT],Net extraordinary revenues (expenses),Profit (loss) for the period [Net income],Costs of employees,Depreciation & Amortization,Other operating items,Interest paid,Cash flow [Net Income before D&A],Added value,EBITDA
2022F,132330.836118,887.652538,117953.373571,13489.810008,129104.648213,7589.918840,261435.484330,48455.404744,102437.390259,100358.714585,...,15756.193127,-13.845995,15742.347131,4757.681710,7679.488372,90661.817549,5248.022178,23421.835503,208082.548712,39735.024916
2023F,130480.427891,-2026.428205,116853.101663,15653.754433,139466.085744,7239.086231,269946.513635,57351.175549,104633.046229,102357.309033,...,15132.594347,90.486816,15223.081163,5365.749783,7800.523871,91693.162158,5681.663369,23023.605034,232880.005930,39976.599796
2024F,127915.573064,-5273.331335,115114.982949,18073.921450,150541.969874,6836.261096,278457.542939,67096.706347,106630.110800,104144.706526,...,14350.069434,207.902302,14557.971736,6025.753801,7920.184911,92573.228587,6145.450526,22778.531978,259694.131233,40656.630965
2025F,124606.368789,-8866.558583,116388.253499,20763.832606,162362.203454,6380.133185,286968.572243,77727.601143,108416.961009,105708.876370,...,14294.446050,338.994654,14295.910857,6739.826138,8039.025222,93298.649884,6640.611749,23458.566361,288605.271013,41870.401326
2026F,122724.106989,-12819.611683,119840.143120,23737.009447,172755.494558,5869.392250,295479.601547,89279.463943,109981.973889,107037.787875,...,14708.824092,484.358063,14710.331362,7510.099170,8157.598533,93866.059100,7168.375137,24138.600744,319693.771662,43084.171686
2027F,126259.066646,-17145.992366,123292.032742,27006.973519,177731.564205,5302.728044,303990.630851,94117.299225,111313.526475,108119.410345,...,15123.202135,644.586722,15124.751867,7968.507607,8276.458573,94272.089282,7729.968791,24818.635128,353039.979572,44297.942046


In [5]:
int(''.join(filter(str.isdigit, '2022F'))) - int(base_year)

1

In [ ]:
def five_year_forecast_financials(df, assumptions, base_year, forecast_years):
    """
    Generates financial forecasts based on provided assumptions and historical data.

    Parameters:
        df (DataFrame): Historical financial data.
        assumptions (dict): Growth assumptions for various financial metrics.
        base_year (str): The base year for the forecast (last historical year).
        forecast_years (list): List of years for which the forecast is to be made.
        
    Returns:
        DataFrame: Forecasted financials including CAGR for the forecast period.
    """
    
    forecast_df = pd.DataFrame(index=df.index, columns=forecast_years)
    last_values = df[base_year].copy()

    for year in forecast_years:
        for metric in last_values.index
            if metric in assumptions.items():
                for values in assumptions.items(metric):
                    base_value = last_values.get(metric, None)
                        if base_value is not None:
                            growth_rate = values['rates'][int(''.join(filter(str.isdigit, year))) - int(base_year) - 1]
                            forecast_value = base_value * (1 + growth_rate)
                            forecast_df.loc[metric, year] = forecast_value
                            last_values[metric] = forecast_value
            else: 
                growth_rate = 
    calculate_cagr(df, forecast_df, base_year, forecast_years)  # Calculate CAGR and add as a column

    return forecast_df


def calculate_historical_cagr(df, base_year, period):
    """
    Calculates the compound annual growth rate (CAGR) for forecasted data.

    Args:
        df (DataFrame): The DataFrame with forecast data.
        forecast_years (list): List of forecast years.
    """
    start_year, end_year = base_year-period, base_year
    cagr_df = pd.DataFrame(index=df.index, columns=np.arange(s,b,1)
    start_year, end_year = base_year-period, base_year
    num_years = period
    df['CAGR'] = ((df[end_year] / df[start_year]) ** (1 / num_years) - 1).fillna(0) * 100


def calculate_cagr(df, forecast_df, base_year, forecast_years, ):
    """
    Calculates the compound annual growth rate (CAGR) for forecasted data.

    Args:
        df (DataFrame): The DataFrame with forecast data.
        forecast_years (list): List of forecast years.
    """
    start_year, end_year = base_year, forecast_years[-1]
    num_years = len(forecast_years)
    forecast_df['CAGR'] = ((forecast_df[end_year] / df[start_year]) ** (1 / num_years) - 1).fillna(0) * 100


In [ ]:
import pandas as pd

def forecast_financials(df, assumptions, base_year, forecast_years, cagr_period=5):
    """
    Generates financial forecasts based on provided assumptions, historical data,
    and rolling CAGR calculations.

    Parameters:
        df (DataFrame): Historical and forecasted financial data.
        assumptions (dict): Explicit growth assumptions for various financial metrics.
        base_year (str): The base year for the forecast (last historical year).
        forecast_years (list): List of years for which the forecast is to be made.
        cagr_period (int): The number of years to use for rolling CAGR calculation.
        
    Returns:
        DataFrame: Forecasted financials including dynamic CAGR for the forecast period.
    """
    
    forecast_df = pd.DataFrame(index=df.index, columns=forecast_years)
    last_values = df[base_year].copy()

    for year_index, year in enumerate(forecast_years):
        # Determine the number of years to look back for CAGR calculation
        start_year = max(int(base_year) - (cagr_period - min(year_index + 1, cagr_period)), int(base_year))
        end_year = int(base_year) + year_index

        for metric in df.index:
            if metric in assumptions and year_index < len(assumptions[metric]['rates']):
                # Use explicit growth rate if available
                growth_rate = assumptions[metric]['rates'][year_index]
            else:
                # Calculate rolling CAGR if not under explicit assumptions or past the available rates
                start_value = df.loc[metric, str(start_year)]
                end_value = df.loc[metric, str(end_year)]
                growth_rate = (end_value / start_value) ** (1 / (end_year - start_year)) - 1 if start_value != 0 else 0
            
            # Calculate forecast value
            base_value = last_values.get(metric, 0)
            forecast_value = base_value * (1 + growth_rate)
            forecast_df.loc[metric, year] = forecast_value
            last_values[metric] = forecast_value

    # Calculate CAGR for the entire forecast period and add as a column
    calculate_cagr(df, forecast_df, base_year, forecast_years)

    return forecast_df

def calculate_cagr(df, forecast_df, base_year, forecast_years):
    """
    Calculates the compound annual growth rate (CAGR) for forecasted data.

    Args:
        df (DataFrame): The DataFrame with historical data.
        forecast_df (DataFrame): The DataFrame with forecast data.
        base_year (str): The starting year for historical data.
        forecast_years (list): List of forecast years.
    """
    start_year, end_year = base_year, forecast_years[-1]
    num_years = len(forecast_years)
    forecast_df['CAGR'] = ((forecast_df[end_year] / df[start_year]) ** (1 / num_years) - 1).fillna(0) * 100

